The exercise solution is revised based on lecture notebook L5

In [1]:
from math import log

DEBUG = True
#DEBUG = False

def p(msg):
    if DEBUG:
        print('.. {}'.format(msg))

def ilog2(x):
    return int(log(x, 2.0))

def encode_unary(vals):
    out = ""

    vs = [vals] if type(vals) == int else vals
    for v in vs:
        out = out + "1" * v
        out = out + "0"

    return out

def encode_binary(vals, width):
    out = ""

    vs = [vals] if type(vals) == int else vals
    for v in vs:
        for i in range(width, 0, -1):
            bit = v >> (i-1) & 0x0001
            if bit > 0:
                out = out + "1"
            else:
              out = out + "0"

    return out


# should check if input val is within the coding range (e.g., >0)
def encode_gamma(vals):
    out_code = []
    for v in vals:
        k_d = ilog2(v)
        k_r = v - (1 << k_d)
        p('val = {}, k_d = {}, k_r = {}'.format(v, k_d, k_r))
        out1 = encode_unary(k_d)
        out2 = encode_binary(k_r, k_d)
        out_code.append([out1, out2])
    return out_code

def encode_delta(vals):
    out_code = []
    for v in vals:
        k_d = ilog2(v)
        k_r = v - (1 << k_d)        
        if k_d == 0:
            out_code.append(['0','0'])
            p('val = 1, k_d = 0, k_r = 0,k_dd = 0,k_dr = 0')
            continue
        k_dd = ilog2(k_d + 1)
        k_dr = k_d + 1 - (1 << k_dd)
        p('val = {}, k_d = {}, k_r = {},k_dd = {},k_dr = {}'.format(v, k_d, k_r,k_dd,k_dr))
        out1 = encode_unary(k_dd) + encode_binary(k_dr,k_dd)
        out2 = encode_binary(k_r, k_d)
        out_code.append([out1, out2])
    return out_code

# out_list is a list of encodings for each value, where each encoding is a list of component
def pp_binary(out_list, width=8, as_string=True):
    if as_string:
        # print as strings of width-byte chunks
        # first, flatten the list
        l = ["".join(v) for v in out_list]
        str = "".join(l)
        n = len(str)
        s = 0
        while s < n:
            e = min(s + width, n)
            print("{} ".format(str[s:e]), end="")
            s += width
        print()
    else:
        for component in out_list:
            print(component)

Now we define the delta decoding method.  As the delta code contains a part of gamma code I revised the decode_gamma method in gamma encoding&decoding exercise solution.

In [4]:
## The method used to decode the delta coding
def decode_gamma(code):
    read_indicator = False
    decode_result = []
    offset_code = ''
    block_code = ''
    length = 0
    position_recorded = False
    bit_pos = 0
    for position,bit in enumerate(code):
        if read_indicator == True and length != 0:
            block_code = block_code + bit
            length = length - 1
            if length == 0:
                read_indicator = False
                block_code = '1' + block_code
                decode_result.append(int(block_code,2))
                block_code = ''
                if position_recorded == False:
                    bit_pos = position + 1
                    position_recorded = True
        else:
            if bit == '1':
                offset_code = offset_code + bit
            elif bit == '0':
                length = len(offset_code)
                if(offset_code == ''):
                    decode_result.append(1)
                    if position_recorded == False:
                        bit_pos = position + 1
                        position_recorded = True
                    
                else:
                    read_indicator = True
                    offset_code = ''
    return decode_result,bit_pos

def decode_delta(code):
    results = []
    length = len(code)
    use_gamma_decoder = True
    gamma_length = 0 ## the number of gamma part
    deviation_length = 0 ## the number of binary part(needed to be minis 1 later)
    delta_code_width = 0 ## the index of whole delta code length
    gamma_code_width = 0 ## the index of last bit of gamma code
    binary = ''
    i = 0
    while True:
        if i == len(code):
            break
        if use_gamma_decoder == True:
            length,gamma_length = decode_gamma(code[i:])
            print(i,length[0],gamma_length)
            delta_code_width = i + gamma_length + length[0] - 1
            gamma_code_width = i + gamma_length
            if length[0] == 1:
                results.append(1)
                i = i + 1
                continue
            i = gamma_code_width
            use_gamma_decoder = False
        print('start is {},end is {}'.format(gamma_code_width,delta_code_width))
        results.append(int('1' + code[gamma_code_width:delta_code_width],2))
        print(results)
        i = delta_code_width
        use_gamma_decoder = True
        
    return results

def generate_raw_code(outlist):
    raw = ''
    for code in outlist:
        if code[0] == '0' and code [1] == '0':
            raw = raw + '0'
        else:
            raw = raw + code[0] + code[1]
    return raw
        

def pre_process(code):
    processed_code = code.replace(" ", "")
    return processed_code

def test_gamma(values):
    out = encode_gamma(values)
    return generate_raw_code(out)

def test_delta(values):
    out = encode_delta(values)
    return generate_raw_code(out)

##code = test_gamma(range(1,16))
code = test_delta(range(1,16))
print(code)
##print(decode_gamma(code))
print(decode_delta(code))

.. val = 1, k_d = 0, k_r = 0,k_dd = 0,k_dr = 0
.. val = 2, k_d = 1, k_r = 0,k_dd = 1,k_dr = 0
.. val = 3, k_d = 1, k_r = 1,k_dd = 1,k_dr = 0
.. val = 4, k_d = 2, k_r = 0,k_dd = 1,k_dr = 1
.. val = 5, k_d = 2, k_r = 1,k_dd = 1,k_dr = 1
.. val = 6, k_d = 2, k_r = 2,k_dd = 1,k_dr = 1
.. val = 7, k_d = 2, k_r = 3,k_dd = 1,k_dr = 1
.. val = 8, k_d = 3, k_r = 0,k_dd = 2,k_dr = 0
.. val = 9, k_d = 3, k_r = 1,k_dd = 2,k_dr = 0
.. val = 10, k_d = 3, k_r = 2,k_dd = 2,k_dr = 0
.. val = 11, k_d = 3, k_r = 3,k_dd = 2,k_dr = 0
.. val = 12, k_d = 3, k_r = 4,k_dd = 2,k_dr = 0
.. val = 13, k_d = 3, k_r = 5,k_dd = 2,k_dr = 0
.. val = 14, k_d = 3, k_r = 6,k_dd = 2,k_dr = 0
.. val = 15, k_d = 3, k_r = 7,k_dd = 2,k_dr = 0
010001001101001010110110101111100000011000001110000101100001111000100110001011100011011000111
0 1 1
1 2 3
start is 4,end is 5
[1, 2]
5 2 3
start is 8,end is 9
[1, 2, 3]
9 3 3
start is 12,end is 14
[1, 2, 3, 4]
14 3 3
start is 17,end is 19
[1, 2, 3, 4, 5]
19 3 3
start is 22,end is 24
[1, 2